# Glacier Gallery

In [ ]:
# https://towardsdatascience.com/exploring-and-visualizing-chicago-transit-data-using-pandas-and-bokeh-part-ii-intro-to-bokeh-5dca6c5ced10
import panel as pn
import os, numpy as np, pandas as pd, cartopy.crs as ccrs, bokeh
import holoviews as hv, geoviews as gv, datashader as ds

from bokeh.models import Legend, LegendItem
from bokeh.models import ColumnDataSource, OpenURL, TapTool, HoverTool, Plot, Range1d, Text, TapTool, CustomJS

from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import get_provider, Vendors
from bokeh.io import output_file, show, output_notebook, reset_output
from bokeh.plotting import figure

hv.extension('bokeh', width=100)

In [ ]:
# load data
df_all = pd.read_csv('https://raw.githubusercontent.com/pyviz-demos/glaciers/master/data/oggm_glacier_explorer.csv')
df_all['latdeg'] = df_all.cenlat
    # short data randomly
df = df_all[::1500]

url = 'https://upload.wikimedia.org/wikipedia/commons/9/95/SantaCruz-PeritoMoreno-P2140146b.jpg'
    # more glacier example pictures
#           'https://www.goodfreephotos.com/albums/argentina/other-argentina/perito-moreno-glacier-argentina.jpg',
#           'https://upload.wikimedia.org/wikipedia/commons/1/18/South_Sawyer_Glacier_48.jpg',
#           'https://upload.wikimedia.org/wikipedia/commons/8/8b/Picture_of_K2.jpg'],
    
df['Coords'] = list(zip(round(df.cenlat,2), round(df.cenlon,2)))
df.loc[:,'images'] = url
df.loc[:,'Type'] = 'Calving glacier'
calving_glacier_text = 'A jégborjadzás a tengerbe \n nyúló gleccserek végeinek, \n illetve jégpajzsok \n peremeinek leszakadása. \n Mivel a jég fajlagos tömege \n csak kb. 0,9 g/cm³, a víz \n megemeli, a hullámzás és a \n tengerjárás pedig mozgatja \n a jégnyelvet. \n A jégnyelvről \n leszakadó és elúszó darabok a \njéghegyek. '
df.loc[:,'glaciertype_text'] = calving_glacier_text

############################################################################
# prepare coordinates  
import math
from ast import literal_eval

def merc(Coords):
    lat = Coords[0]
    lon = Coords[1]

    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale    
    return (x, y)

df.loc[:, 'coords_x'] = df['Coords'].apply(lambda x: merc(x)[0])
df.loc[:, 'coords_y'] = df['Coords'].apply(lambda x: merc(x)[1])

############################################################################
# empty list
empty1 = [''] * len(df)
empty2 = [''] * len(df)

############################################################################
source = ColumnDataSource(
    data=dict(
                        x=list(df['coords_x']), 
                        y=list(df['coords_y']),
                        Area=list(df['area_km2']),
                        prcp=list(df['avg_prcp']),
                        gtype=list(df['glacier_type']),
                        img=list(df['images']),
                        real_gltype=list(df['Type']),
                        gltype_text=list(df['glaciertype_text']),
                        empty1=empty1,
                        empty2=empty2,
                        RGI_id=list(df['rgi_id']))
                       )

############################################################################
# Hover
TOOLTIPS = """
    <div>
        <div>
            <img
                src="@img" height="300" alt="Bild" width="400"
                style="float: left; margin: 0px 50px 50px 0px;"
                border="2"
            ></img>
        </div>
        <div>
            <span style="font-size: 17px; font-weight: bold;">Glacier Name</span>
            <span style="font-size: 15px; color: #966;">[$index]</span>
        </div>
    </div>
"""

hover = HoverTool()
hover.tooltips = TOOLTIPS

############################################################################
p = figure(x_axis_type="mercator", 
           y_axis_type="mercator",
            plot_width = 1000,
           plot_height = 650,
            background='black',
           tools=[hover, 'tap', 'wheel_zoom','save'])

############################################################################
# Design figure
p.xaxis.visible = False
p.yaxis.visible = False
p.xgrid.visible = False
p.ygrid.visible = False
p.xgrid.grid_line_color = None

############################################################################
# add map
tile_provider = get_provider(Vendors.CARTODBPOSITRON)
p.add_tile(tile_provider)

############################################################################
# add glacier dots
p.circle(x='x',
         y='y', 
         source=source,
         size=10,
         #line_color="blue", 
         fill_color="blue",
         fill_alpha=0.3)

############################################################################
# build a text box with information about glacier types
def construct_text_box(source): 
    # Plot and axes                                                             
    xdr = Range1d(0, 1220)                                                       
    ydr = Range1d(0, 1120)                                                       

    plot = Plot(                                                                
        x_range=xdr, y_range=ydr,                                   
        plot_height=320, min_border=0, 
        plot_width=260
    )                                                                           
    # Add the writing 
    # textstyle:    https://bokeh.pydata.org/en/latest/docs/user_guide/styling.html#fill-properties
    country1 = Text(x=10, y=100, text='empty1', text_color="#85a9c1", text_alpha=1.0, text_font='helvetica', text_font_size='10pt')# angle, angle_units, js_event_callbacks, js_property_callbacks, name, subscribed_events, 
                    #tags, text_align, text_baseline, text_color, text_font, text_font_style, text_line_height, x, x_offset, y, y_offset)
    country2 = Text(x=10, y=1000, text='empty2', text_color="#326a86", text_alpha=0.2, text_font='helvetica', text_font_size='14pt')
    plot.add_glyph(source, country1)
    plot.add_glyph(source, country2)    

    return plot

JS_CODE ="""
    // Get index of current selected datapoint
    sel_point_index = cb_data.source.attributes.selected["1d"]["indices"][0];     

    /* replace all name_display values with the name_label
       value of currently selected point */
    for (i=0; i < cb_data.source.data.real_gltype.length; i++) {
        cb_data.source.data.empty2[i] = cb_data.source.data.real_gltype[sel_point_index];
        }       
    cb_data.source.change.emit();
    
    for (i=0; i < cb_data.source.data.real_gltype.length; i++) {
        cb_data.source.data.empty1[i] = cb_data.source.data.gltype_text[sel_point_index];
        }      
     cb_data.source.change.emit();

    """

newtaptool = p.select(type=TapTool)
newtaptool.callback = CustomJS(code=JS_CODE)
text_box = construct_text_box(source)

############################################################################
## legend

#li1 = LegendItem(label='Rock glaciers', renderers=[plot.renderers[2]])
li2 = LegendItem(label='Glaciers', renderers=[p.renderers[1]])
#legend1 = Legend(items=[li1, li2], location='top_right', click_policy='hide')
legend1 = Legend(items=[li2], location='top_right', click_policy='hide')
p.add_layout(legend1)


############################################################################
# Design Application
# Panel
title       = '<div style="font-size:35px">Glacier Gallery</div>'
instruction = 'The blue areas mark glaciers. Move the mouse over the blue dots to show pictures and information. Click on it and get information about the glacier type.<br>'
oggm_logo   = '<a href="http://edu.oggm.org"><img src="https://raw.githubusercontent.com/zschirmeister/glacier-gallery/master/oggm_loupe.png" width=220></a>'
pn_logo     = '<a href="https://panel.pyviz.org"><img src="http://panel.pyviz.org/_static/logo_stacked.png" width=140></a>'

header = pn.Row(pn.Pane(oggm_logo),  pn.layout.Spacer(width=30), 
                pn.Column(pn.Pane(title, width=400), pn.Pane(instruction, width=500)),
                pn.layout.HSpacer(), pn.layout.Spacer(height=20), 
                pn.Pane(pn_logo, width=140))

# Put plot and text together and show it
pn.Column(header, pn.Spacer(height=15), pn.Row(p, pn.Spacer(width=5), text_box), width_policy='max', height_policy='max').servable()

In [ ]:
df